In [32]:
import numba as nb

In [33]:
import squidpy as sq
import scanpy as sc
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, isspmatrix_csr

In [34]:
import liana as li

In [35]:
from matplotlib.pyplot import hist

In [36]:
from liana.method._global_lr_pipe import _global_lr_pipe
from liana.method.sp._spatialdm import _get_ordered_matrix, _standardize_matrix

In [37]:
from liana.method.sp._bivariate_funs import _masked_coexpressions, _vectorized_correlations, _vectorized_wcosine, _vectorized_jaccard

In [38]:
from liana.method.sp._spatialdm import _local_morans, _standardize_matrix

In [ ]:
# # scHOT data
# counts = pd.read_csv("data/counts_mat.csv")
# weights = pd.read_csv("data/weight_mat.csv")
# var = pd.DataFrame(counts[['Unnamed: 0']]).set_index('Unnamed: 0')
# var.index.name = None
# adata = sc.AnnData(X=csr_matrix(counts.drop(counts.columns[0], axis=1), dtype=np.float32).T, var=var)
# adata.obsm['proximity'] = csr_matrix(weights)

In [ ]:
# scHOT data test
adata = sc.read_h5ad("test_spatial.h5ad")
resource = pd.DataFrame({"ligand":["Dnm1l", "Arrb1", "Igf2", "Dnm1l"], "receptor":["Gucy1b3", "Mtor", "Tuba1a", "Fam63b"]})
dist = adata.obsm['proximity']

In [ ]:
n_perm = 100
seed = 0

In [ ]:
# full visium slide
# load the pre-processed dataset
img = sq.datasets.visium_hne_image()
adata = sq.datasets.visium_hne_adata()

li.mt.get_spatial_proximity(adata=adata, parameter=200, bypass_diagonal=False, cutoff=0.1, family="exponential")
dist = adata.obsm['proximity']

my_p = li.pl.proximity_plot(adata, idx=100)
resource = li.resource.select_resource("mouseconsensus")

In [ ]:
temp, lr_res, ligand_pos, receptor_pos = _global_lr_pipe(adata=adata,
                                                         resource=resource,
                                                         expr_prop=0.05,
                                                         use_raw=False,
                                                         verbose=True,
                                                         layer=None,
                                                         _key_cols=['ligand_complex', 'receptor_complex'],
                                                         _complex_cols=['ligand_means', 'receptor_means'],
                                                         _obms_keys=['proximity'],
                                                         resource_name=None
                                                         )

In [ ]:
# lr_res = lr_res.head(50)

In [ ]:
x_mat = _get_ordered_matrix(temp.X, ligand_pos, lr_res.ligand).A.astype(np.float64).T
y_mat = _get_ordered_matrix(temp.X, receptor_pos, lr_res.receptor).A.astype(np.float64).T

In [ ]:
lr_res.head()

In [ ]:
dist = adata.obsm['proximity']
weight = dist.A.astype(np.float64)

In [ ]:
%%time
masked_pc = _masked_coexpressions(x_mat, y_mat, weight, weight_thr=0.0, method=0)

In [ ]:
%%time
masked_sp = _masked_coexpressions(x_mat, y_mat, weight, weight_thr=0.0, method=1)

In [ ]:
masked_pc

In [ ]:
masked_sp

Fully-vectorized

In [39]:
import squidpy as sq
import scanpy as sc
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, isspmatrix_csr

In [40]:
import liana as li

In [41]:
from matplotlib.pyplot import hist
from scipy.stats import spearmanr, pearsonr

In [42]:
from liana.method._global_lr_pipe import _global_lr_pipe
from liana.method.sp._spatialdm import _get_ordered_matrix, _standardize_matrix

In [43]:
# ligand-receptor mats
ligand_mat = _get_ordered_matrix(temp.X, ligand_pos, lr_res.ligand).T
receptor_mat = _get_ordered_matrix(temp.X, receptor_pos, lr_res.receptor).T

In [44]:
dist = adata.obsm['proximity']

In [45]:
local_pc = _vectorized_correlations(x_mat = ligand_mat.A, y_mat=receptor_mat.A, dist=dist, method="pearson")

In [46]:
local_sp = _vectorized_correlations(x_mat = ligand_mat.A, y_mat=receptor_mat.A, dist=dist, method="spearman")

In [47]:
local_cos = _vectorized_wcosine(x_mat = ligand_mat.A, y_mat=receptor_mat.A, dist=dist)

In [48]:
local_jc = _vectorized_jaccard(x_mat = ligand_mat.A, y_mat=receptor_mat.A, dist=dist)

In [66]:
ligand_mat = _get_ordered_matrix(temp.X, ligand_pos, lr_res.ligand).T
receptor_mat = _get_ordered_matrix(temp.X, receptor_pos, lr_res.receptor).T

In [67]:
# if spatial DM: standardize & calculate norm factor
ligand_mat = _standardize_matrix(ligand_mat, local=True)
receptor_mat = _standardize_matrix(receptor_mat, local=True)

In [68]:
norm_factor = temp.obsm['proximity'].shape[0] / temp.obsm['proximity'].sum()
dist = csr_matrix(norm_factor * temp.obsm['proximity'])

In [69]:
local_r = _local_morans(x_mat = ligand_mat, y_mat=receptor_mat, dist=dist)

In [76]:
local_r

array([[-1.31601786,  0.09108513,  0.19667045, ...,  0.01102248,
         0.02195861,  0.01416014],
       [-1.41772597,  0.10259609,  0.32585438, ...,  0.00552196,
        -0.02893247, -0.83329622],
       [-1.66266239,  0.52212755,  1.58267848, ...,  0.21773563,
         0.30884862, -1.12790739],
       [ 0.11439858, -0.01571357,  0.04996999, ..., -0.10334864,
        -0.0432732 , -0.00971188]])

In [ ]:
local_r

In [ ]:
from liana.method.sp._spatial_utils import _local_permutation_pvals

In [ ]:
%%time
local_pc_pvals = _local_permutation_pvals(x_mat = ligand_mat.A, 
                                          y_mat = receptor_mat.A, 
                                          local_truth=local_pc,
                                          local_fun=_vectorized_correlations,
                                          dist=dist, 
                                          n_perm=n_perm, 
                                          positive_only=False,
                                          seed=seed)

In [ ]:
local_pc_pvals

In [ ]:
local_sp = _vectorized_correlations(x_mat = ligand_mat.A, y_mat=receptor_mat.A, dist=dist, method="spearman")

In [ ]:
local_sp_pvals = _local_permutation_pvals(x_mat = ligand_mat.A, 
                                          y_mat = receptor_mat.A, 
                                          local_truth=local_sp,
                                          local_fun=_vectorized_correlations,
                                          dist=dist, 
                                          n_perm=n_perm, 
                                          positive_only=False,
                                          seed=seed,
                                          method="spearman"
                                          )

In [ ]:
local_sp_pvals.shape

In [ ]:
spearmanr(local_sp_pvals[1,:], local_pc_pvals[1,:])

Global summary of the local scores:

In [ ]:
lr_res.loc[:,['pearson_mean','pearson_sd']] = np.vstack([np.mean(local_pc, axis=1), np.std(local_pc, axis=1)]).T

In [ ]:
lr_res.sort_values(by='pearson_mean', ascending=False)

Plot to check the distribution of the local scores:

In [ ]:
from liana.method.sp._spatialdm import spatialdm

In [ ]:
spatialdm(adata=adata, resource_name="MouseConsensus", pvalue_method='analytical', verbose=True, use_raw=False)

In [ ]:
local_pvals = li.ut.obsm_to_adata(adata, obsm_key='local_pvals')

In [ ]:
local_r = li.ut.obsm_to_adata(adata, obsm_key='local_r')

In [ ]:
sc.pl.spatial(local_pvals, color=['Sema4d&Plxnb3'])

In [ ]:
sc.pl.spatial(local_r, color=['Sema4d&Plxnb3'])

In [ ]:
adata.obsm['local_r']

In [ ]:
def embed_score_to_adata(adata, score, score_name):
    score =  pd.DataFrame(score,
                          columns=adata.obsm['local_r'].columns,
                          index=adata.obsm['local_r'].index)
    adata.obsm[score_name] = score

In [ ]:
# Masked Pearson correlation
embed_score_to_adata(adata, masked_pc, 'masked_pc')

# Vectorized Pearson correlation
embed_score_to_adata(adata, local_pc.T, 'local_pc')

In [ ]:
# Masked Spearman correlation
embed_score_to_adata(adata, masked_sp, 'masked_sp')
# Vectorized Spearman correlation
embed_score_to_adata(adata, local_sp.T, 'local_sp')

In [ ]:
# Vectorized Jaccard index
embed_score_to_adata(adata, local_jc.T, 'local_jc')

In [ ]:
# Vectorized Cosine similarity
embed_score_to_adata(adata, local_cos.T, 'local_cos')

Plot to check the distribution of the local scores:

In [ ]:
sc.pl.spatial(adata, color=['Sema4d', 'Plxnb3'], use_raw=False)

In [ ]:
local_r = li.ut.obsm_to_adata(adata, obsm_key='local_r')
sc.pl.spatial(local_r, color=['Sema4d&Plxnb3'])

In [ ]:
sc.pl.spatial(local_pvals, color=['Sema4d&Plxnb3'], cmap='viridis_r')

Pearson

In [ ]:
masked_pc = li.ut.obsm_to_adata(adata, obsm_key='masked_pc')
sc.pl.spatial(masked_pc, color=['Sema4d&Plxnb3'])

In [ ]:
np.average(masked_pc.X)

In [ ]:
local_pc = li.ut.obsm_to_adata(adata, obsm_key='local_pc')
sc.pl.spatial(local_pc, color=['Sema4d&Plxnb3'])

In [ ]:
np.average(local_pc.X)

In [ ]:
local_sp = li.ut.obsm_to_adata(adata, obsm_key='local_sp')
sc.pl.spatial(local_sp, color=['Sema4d&Plxnb3'])

In [ ]:
masked_sp = li.ut.obsm_to_adata(adata, obsm_key='masked_sp')
sc.pl.spatial(masked_sp, color=['Sema4d&Plxnb3'])

Jaccard

In [ ]:
local_jc = li.ut.obsm_to_adata(adata, obsm_key='local_jc')
sc.pl.spatial(local_jc, color=['Sema4d&Plxnb3'])

Cosine

In [ ]:
local_cos = li.ut.obsm_to_adata(adata, obsm_key='local_cos')
sc.pl.spatial(local_cos, color=['Sema4d&Plxnb3'])

Pvals

In [ ]:
embed_score_to_adata(adata, local_pc_pvals.T, 'local_pc_pvals')
local_pc_pvals = li.ut.obsm_to_adata(adata, obsm_key='local_pc_pvals')

In [ ]:
sc.pl.spatial(local_pc_pvals, color=['Sema4d&Plxnb3'], cmap='viridis_r')

In [ ]:
# LASSO regression on each spot?